In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [2]:
# Load the Flowers-102 dataset
(ds_train, ds_test), ds_info = tfds.load('tf_flowers', split=['train[:80%]', 'train[80%:]'], as_supervised=True, with_info=True)

# Define a function to resize images
def resize_image(image, label):
    image = tf.image.resize(image, (32, 32))
    return image, label

# Apply resizing to the dataset
ds_train = ds_train.map(resize_image)
ds_test = ds_test.map(resize_image)

# Normalize pixel values to be in the range [0, 1]
ds_train = ds_train.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))
ds_test = ds_test.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))

# Shuffle and batch the datasets
ds_train = ds_train.shuffle(10000).batch(64)
ds_test = ds_test.batch(64)

# Convert labels to categorical format
def categorical_labels(images, labels):
    return images, tf.one_hot(labels, depth=102)  # 102 classes in Flowers-102 dataset

ds_train = ds_train.map(categorical_labels)
ds_test = ds_test.map(categorical_labels)

# Extracting data from the TensorFlow dataset into lists
X_train_list, Y_train_list = [], []
for images, labels in ds_train:
    X_train_list.append(images.numpy())
    Y_train_list.append(labels.numpy())

X_test_list, Y_test_list = [], []
for images, labels in ds_test:
    X_test_list.append(images.numpy())
    Y_test_list.append(labels.numpy())

# Convert lists to numpy arrays
X_train = np.concatenate(X_train_list, axis=0)
Y_train = np.concatenate(Y_train_list, axis=0)
X_test = np.concatenate(X_test_list, axis=0)
Y_test = np.concatenate(Y_test_list, axis=0)

print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
print("X_test shape:", X_test.shape)
print("Y_test shape:", Y_test.shape)


X_train shape: (2936, 32, 32, 3)
Y_train shape: (2936, 102)
X_test shape: (734, 32, 32, 3)
Y_test shape: (734, 102)


In [3]:
from tensorflow import keras
model = keras.models.load_model(r'C:\Users\sadia_tisha1\Desktop\CODE\Flowers102_32_model.h5')

In [8]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
    
num_classes = 102  # Update with the actual number of classes in your target data
x = model.layers[-4].output  # Access the last 4th layer from the end
x = GlobalAveragePooling2D()(x)
output = Dense(num_classes, activation='softmax')(x)

# Create the new model with the updated head
new_model = keras.models.Model(inputs=model.input, outputs=output)

# Compile the model
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
new_model.fit(X_train, Y_train, batch_size=64, epochs=50, validation_split=0.2)

Epoch 1/50
37/37 [==============================] - 168s 4s/step - loss: 1.2214 - accuracy: 0.5967 - val_loss: 3816.8738 - val_accuracy: 0.2041
Epoch 2/50
37/37 [==============================] - 153s 4s/step - loss: 0.7221 - accuracy: 0.7509 - val_loss: 210.0349 - val_accuracy: 0.2041
Epoch 3/50
37/37 [==============================] - 93s 2s/step - loss: 0.5986 - accuracy: 0.8164 - val_loss: 107.6483 - val_accuracy: 0.2602
Epoch 4/50
37/37 [==============================] - 87s 2s/step - loss: 0.5750 - accuracy: 0.8254 - val_loss: 45.4498 - val_accuracy: 0.4184
Epoch 5/50
37/37 [==============================] - 86s 2s/step - loss: 0.3684 - accuracy: 0.8795 - val_loss: 6.3547 - val_accuracy: 0.5408
Epoch 6/50
37/37 [==============================] - 87s 2s/step - loss: 0.3012 - accuracy: 0.9084 - val_loss: 13.1015 - val_accuracy: 0.5357
Epoch 7/50
37/37 [==============================] - 86s 2s/step - loss: 0.5086 - accuracy: 0.8765 - val_loss: 2.3858 - val_accuracy: 0.5289
Epoch 8/5

In [9]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Make predictions on the test dataset
predictions = new_model.predict(ds_test)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Manually generate ground truth labels for evaluation
true_labels = []
for _, label in ds_test:
    true_labels.extend(label.numpy())

true_labels = np.argmax(true_labels, axis=1)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Assuming you have test_loss and test_accuracy available
# If not, you'll need to calculate them based on your model and test dataset
print(f"Test Accuracy: {accuracy}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Test Accuracy: 0.4564032697547684
Accuracy: 0.4564032697547684
F1 Score: 0.46374841503180314
